<a href="https://colab.research.google.com/github/MattiaPOLI/DR14/blob/master/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!rm -rf DR14
!git clone https://github.com/MattiaPOLI/DR14.git

Cloning into 'DR14'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 260 (delta 68), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (260/260), 3.88 MiB | 7.15 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from DR14 import CommonTools
import IPython
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
import pandas as pd
import numpy as np
import io

In [35]:
#original dataset

x_orig, y_orig = CommonTools.get_raw_dataset()

trainSet, test, labelTrainSet, labelTest = train_test_split(x_orig, y_orig, train_size = 0.7, test_size = 0.3, random_state = 40)

numberTrees = [50, 100, 200, 300, 350, 400, 500]
numberFeatures = [2, 3, 4, 5]
accuracy = np.zeros(shape = (len(numberTrees), len(numberFeatures)))

for i in range(len(numberTrees)):
  for j in range(len(numberFeatures)):
    RFC = RandomForestClassifier(n_estimators = numberTrees[i], random_state=0, oob_score = True,
                                max_features = numberFeatures[j], bootstrap = True)
    RFC.fit(trainSet, labelTrainSet)
    accuracy[i][j] = RFC.oob_score_
    
trace = go.Heatmap(
    z = accuracy,
    colorscale = "Blues",
    reversescale = True
)
layout = go.Layout(
    title = "OOB error",
    xaxis = dict(
        tickmode = "array",
        tickvals = np.linspace(0, len(numberFeatures), num = len(numberFeatures) + 1),       
        ticktext = ["2", "3", "4", "5"],
        title = "N° Features"      
    ),
    yaxis = dict(
        tickmode = "array",
        tickvals = np.linspace(0, len(numberTrees), num = len(numberTrees) + 1),
        ticktext = ["50", "100", "200", "300", "350", "400", "500"],
        title = "N° Tree"
    )
)
data = [trace]
figure = go.Figure(data = data, layout = layout)
CommonTools.enable_plotly_in_cell()
plotly.offline.iplot(figure, filename="rbfAccuracy") 

In [0]:
#fare la precisione

In [34]:
#most relevant

x_orig, y_orig = CommonTools.get_most_relevant_dataset()

trainSet, test, labelTrainSet, labelTest = train_test_split(x_orig, y_orig, train_size = 0.7, test_size = 0.3, random_state = 40)

numberTrees = [50, 100, 200, 300, 350, 400, 500]
numberFeatures = [2, 3, 4, 5]
accuracy = np.zeros(shape = (len(numberTrees), len(numberFeatures)))

for i in range(len(numberTrees)):
  for j in range(len(numberFeatures)):
    RFC = RandomForestClassifier(n_estimators = numberTrees[i], random_state=0, oob_score = True,
                                max_features = numberFeatures[j], bootstrap = True)
    RFC.fit(trainSet, labelTrainSet)
    accuracy[i][j] = RFC.oob_score_
    
trace = go.Heatmap(
    z = accuracy,
    colorscale = "Blues",
    reversescale = True
)
layout = go.Layout(
    title = "OOB error",
    xaxis = dict(
        tickmode = "array",
        tickvals = np.linspace(0, len(numberFeatures), num = len(numberFeatures) + 1),       
        ticktext = ["2", "3", "4", "5"],
        title = "N° Features"      
    ),
    yaxis = dict(
        tickmode = "array",
        tickvals = np.linspace(0, len(numberTrees), num = len(numberTrees) + 1),
        ticktext = ["50", "100", "200", "300", "350", "400", "500"],
        title = "N° Tree"
    )
)
data = [trace]
figure = go.Figure(data = data, layout = layout)
CommonTools.enable_plotly_in_cell()
plotly.offline.iplot(figure, filename="rbfAccuracy") 

In [36]:
#most meaningful analysis

x_orig, y_orig = CommonTools.get_meaningful_dataset()

trainSet, test, labelTrainSet, labelTest = train_test_split(x_orig, y_orig, train_size = 0.7, test_size = 0.3, random_state = 40)

numberTrees = [50, 100, 200, 300, 350, 400, 500]
numberFeatures = [2, 3, 4, 5]
accuracy = np.zeros(shape = (len(numberTrees), len(numberFeatures)))

for i in range(len(numberTrees)):
  for j in range(len(numberFeatures)):
    RFC = RandomForestClassifier(n_estimators = numberTrees[i], random_state=0, oob_score = True,
                                max_features = numberFeatures[j], bootstrap = True)
    RFC.fit(trainSet, labelTrainSet)
    accuracy[i][j] = RFC.oob_score_
    
trace = go.Heatmap(
    z = accuracy,
    colorscale = "Blues",
    reversescale = True
)
layout = go.Layout(
    title = "OOB error",
    xaxis = dict(
        tickmode = "array",
        tickvals = np.linspace(0, len(numberFeatures), num = len(numberFeatures) + 1),       
        ticktext = ["2", "3", "4", "5"],
        title = "N° Features"      
    ),
    yaxis = dict(
        tickmode = "array",
        tickvals = np.linspace(0, len(numberTrees), num = len(numberTrees) + 1),
        ticktext = ["50", "100", "200", "300", "350", "400", "500"],
        title = "N° Tree"
    )
)
data = [trace]
figure = go.Figure(data = data, layout = layout)
CommonTools.enable_plotly_in_cell()
plotly.offline.iplot(figure, filename="rbfAccuracy") 

In [37]:
#most meaningful analysis

x_orig, y_orig = CommonTools.get_pca_dataset()

trainSet, test, labelTrainSet, labelTest = train_test_split(x_orig, y_orig, train_size = 0.7, test_size = 0.3, random_state = 40)

numberTrees = [50, 100, 200, 300, 350, 400, 500]
numberFeatures = [2, 3, 4, 5]
accuracy = np.zeros(shape = (len(numberTrees), len(numberFeatures)))

for i in range(len(numberTrees)):
  for j in range(len(numberFeatures)):
    RFC = RandomForestClassifier(n_estimators = numberTrees[i], random_state=0, oob_score = True,
                                max_features = numberFeatures[j], bootstrap = True)
    RFC.fit(trainSet, labelTrainSet)
    accuracy[i][j] = RFC.oob_score_
    
trace = go.Heatmap(
    z = accuracy,
    colorscale = "Blues",
    reversescale = True
)
layout = go.Layout(
    title = "OOB error",
    xaxis = dict(
        tickmode = "array",
        tickvals = np.linspace(0, len(numberFeatures), num = len(numberFeatures) + 1),       
        ticktext = ["2", "3", "4", "5"],
        title = "N° Features"      
    ),
    yaxis = dict(
        tickmode = "array",
        tickvals = np.linspace(0, len(numberTrees), num = len(numberTrees) + 1),
        ticktext = ["50", "100", "200", "300", "350", "400", "500"],
        title = "N° Tree"
    )
)
data = [trace]
figure = go.Figure(data = data, layout = layout)
CommonTools.enable_plotly_in_cell()
plotly.offline.iplot(figure, filename="rbfAccuracy") 